In [1]:
import pandas as pd

# Load the Excel files
file_path_1 = '附件1.xlsx'
file_path_2 = '附件2.xlsx'

# Load both Excel files
data1 = pd.read_excel(file_path_1)
data2_sheet1 = pd.read_excel(file_path_2, sheet_name=0)
data2_sheet2 = pd.read_excel(file_path_2, sheet_name=1)

# Step 1: Create a mapping from '附件1' for 地块名称 to 地块类型
land_type_mapping = data1[['地块名称', '地块类型']].set_index('地块名称').to_dict()['地块类型']

# Step 2: Add 地块类型 to '附件2' based on the 地块名称
data2_sheet1['地块类型'] = data2_sheet1['地块名称'].map(land_type_mapping)

# Step 3: Create a multi-index mapping for 作物编号, 地块类型, and 种植季次 to 亩产量
land_type_mapping1 = data2_sheet2.set_index(['作物编号', '地块类型', '种植季次']).to_dict()['亩产量/斤']

# Step 4: Use apply() to apply the multi-index mapping to the corresponding rows in data2_sheet1
data2_sheet1['亩产量'] = data2_sheet1.apply(
    lambda row: land_type_mapping1.get((row['作物编号'], row['地块类型'], row['种植季次'])), axis=1
)

# First, define the seasonal periods based on the 地块类型 and 种植季次 as a mapping
def get_season_period(地块类型, 种植季次):
    if 地块类型 == '水浇地':
        if 种植季次 == '第一季':
            return '3月-6月'
        elif 种植季次 == '第二季':
            return '7月-10月'
    elif 地块类型 == '普通大棚':
        if 种植季次 == '第一季':
            return '5月-9月'
        elif 种植季次 == '第二季':
            return '9月-次年4月'
    elif 地块类型 == '智慧大棚':
        if 种植季次 == '第一季':
            return '3月-7月'
        elif 种植季次 == '第二季':
            return '8月-次年2月'
    return '未知'

# Apply the function to add a '种植时间段' column to data2_sheet1
data2_sheet1['种植时间段'] = data2_sheet1.apply(
    lambda row: get_season_period(row['地块类型'], row['种植季次']), axis=1)

# Now, calculate the '预期销售量' (Expected sales volume = 种植面积 * 亩产量)
data2_sheet1['预期销售量'] = data2_sheet1['种植面积/亩'] * data2_sheet1['亩产量']

# Group by 作物名称 and 种植季次, and sum the '预期销售量'
seasonal_sales = data2_sheet1.groupby(['作物名称', '种植季次'])['预期销售量'].sum().reset_index()

data2_sheet1

,地块名称,作物编号,作物名称,作物类型,种植面积/亩,种植季次,地块类型,亩产量,种植时间段,预期销售量
0,A1,6,小麦,粮食,80.0,单季,平旱地,800,未知,64000.0
1,A2,7,玉米,粮食,55.0,单季,平旱地,1000,未知,55000.0
2,A3,7,玉米,粮食,35.0,单季,平旱地,1000,未知,35000.0
3,A4,1,黄豆,粮食（豆类）,72.0,单季,平旱地,400,未知,28800.0
4,A5,4,绿豆,粮食（豆类）,68.0,单季,平旱地,350,未知,23800.0
...,...,...,...,...,...,...,...,...,...,...
82,F3,28,小青菜,蔬菜,0.3,第二季,智慧大棚,3600,8月-次年2月,1080.0
83,F3,30,生菜,蔬菜,0.3,第二季,智慧大棚,4500,8月-次年2月,1350.0
84,F4,19,芸豆,蔬菜（豆类）,0.6,第一季,智慧大棚,3600,3月-7月,2160.0
85,F4,34,芹菜,蔬菜,0.3,第二季,智慧大棚,6000,8月-次年2月,1800.0


In [ ]:
第一种情况

In [15]:
import pandas as pd
from pulp import LpMaximize, LpProblem, LpVariable, lpSum, LpBinary

# 读取Excel文件中的数据
file1_path = '附件1.xlsx'
file2_path = '附件2_1.xlsx'

# 读取每个工作表的数据
file1_data1 = pd.read_excel(file1_path, sheet_name='乡村的现有耕地')
file1_data2 = pd.read_excel(file1_path, sheet_name='乡村种植的农作物')
file2_data1 = pd.read_excel(file2_path, sheet_name='Sheet1')
file2_data2 = pd.read_excel(file2_path, sheet_name='Sheet2')

# 清理列名
file1_data1.columns = ['land_name', 'land_type', 'land_area', 'description']
file1_data2.columns = ['crop_id', 'crop_name', 'crop_type', 'suitable_land', 'description']
file2_data1.columns = ['land_name', 'crop_id', 'crop_name', 'crop_type', 'planting_area', 
                      'planting_season', 'land_type', 'yield_per_mu', 'planting_period', 'expected_sales']
file2_data2.columns = ['serial_no', 'crop_id', 'crop_name', 'land_type', 'planting_season', 
                       'yield_per_mu', 'planting_cost_per_mu', 'sales_price_per_unit']

# 处理价格区间，将价格区间转换为均值
def parse_price_range(price_str):
    if isinstance(price_str, str) and '-' in price_str:
        low, high = map(float, price_str.split('-'))
        return (low + high) / 2
    return float(price_str)

file2_data2['average_price'] = file2_data2['sales_price_per_unit'].apply(parse_price_range)

# 使用 crop_id 作为键创建字典
crop_prices = dict(zip(file2_data2['crop_id'], file2_data2['average_price']))
crop_yields = dict(zip(file2_data1['crop_id'], file2_data1['yield_per_mu']))
expected_sales = dict(zip(file2_data1['crop_id'], file2_data1['expected_sales']))
cost = dict(zip(file2_data2['crop_id'], file2_data2['planting_cost_per_mu']))

# 创建土地面积的字典
land_areas = dict(zip(file1_data1['land_name'], file1_data1['land_area']))

# 定义每个地块类型对应的种植季节数
season_info = {
    '平旱地': 1,  # 每年只能种一季
    '梯田': 1,    # 每年只能种一季
    '山坡地': 1,  # 每年只能种一季
    '水浇地': 2,  # 每年可以种一季或两季
    '大棚': 2     # 每年可以种两季
}

# 根据 land_type 决定每块地每年可以种的季节数
seasons = {row['land_name']: season_info.get(row['land_type'], 1) for idx, row in file1_data1.iterrows()}

# 创建一个字典，存储每个作物可以种植的土地类型
crop_suitable_land = {}
for idx, row in file1_data2.iterrows():
    if pd.notna(row['suitable_land']):
        suitable_land_types = row['suitable_land'].split('\n')
        crop_suitable_land[row['crop_id']] = suitable_land_types

# 定义年份范围
years = list(range(2024, 2031))  # 从2024年到2030年，共7年

# 定义优化模型
model = LpProblem("Crop_Allocation_Optimization", LpMaximize)

# 决策变量的定义：种植面积不能为负
x = LpVariable.dicts("plant_area", 
                     [(i, j, k, t) for i in land_areas for j in crop_yields for k in years for t in range(1, seasons[i] + 1)], 
                     lowBound=0)  # lowBound=0 保证种植面积为非负数

# 二值变量 y 表示是否种植该作物
y = LpVariable.dicts("plant_decision", 
                     [(i, j, k, t) for i in land_areas for j in crop_yields for k in years for t in range(1, seasons[i] + 1)], 
                     cat=LpBinary)  # y 变量是二进制变量

# 目标函数：最大化收益（滞销部分不计入）
model += lpSum(crop_prices[j] * crop_yields[j] * x[i, j, k, t] - x[i, j, k, t] * cost[j]
               for i in land_areas for j in crop_yields for k in years for t in range(1, seasons[i] + 1))

# 约束1：耕地面积约束 - 调整每块地每年每季种植面积的约束，确保每年不超过可用面积
for i in land_areas:
    for k in years:
        # 每年总种植面积不能超过该地块的总可用面积
        model += lpSum(x[i, j, k, t] for j in crop_yields for t in range(1, seasons[i] + 1)) <= land_areas[i]

# 手动输入豆类作物的 crop_id 列表
bean_crops = [1, 2, 3, 4, 5, 17, 18, 19]

# 约束2：每三年内至少种植一次豆类作物
for i in land_areas:
    # 确保每块地在每三年内至少种植一次豆类作物
    for k_start in range(2024, 2031, 3):  # 从2024年开始，步长为3年
        model += lpSum(y[i, j, k, t] for j in bean_crops for k in range(k_start, min(k_start+3, 2031))
                       for t in range(1, seasons[i] + 1)) >= 1

# 约束3：作物产量不得超过预期销售量
for i in land_areas:
    for j in crop_yields:
        for k in years:
            for t in range(1, seasons[i] + 1):
                model += crop_yields[j] * x[i, j, k, t] <= expected_sales[j]

# 约束4：同一地块不能在连续季节种植相同作物
for i in land_areas:
    for j in crop_yields:
        for k in years:
            for t in range(1, seasons[i]):  # 检查同一年不同季节
                model += y[i, j, k, t] + y[i, j, k, t+1] <= 1

# 约束5：最小种植面积约束（当 y[i, j, k, t] == 1，即种植某种作物时，才要求种植面积大于最小面积）
for i in land_areas:
    for j in crop_yields:
        for k in years:
            for t in range(1, seasons[i] + 1):
                # 如果种植某作物 (y[i, j, k, t] == 1)，种植面积必须 >= 0.5 * land_areas[i]
                model += x[i, j, k, t] >= 0.5 * land_areas[i] * y[i, j, k, t]

# 新增约束6：作物只能种植在适合的土地类型上
for i in land_areas:
    for j in crop_yields:
        land_type = file1_data1.loc[file1_data1['land_name'] == i, 'land_type'].values[0]
        # 先检查 crop_id 是否存在于 crop_suitable_land 字典中
        if j in crop_suitable_land:
            # 如果当前地块类型不适合该作物种植，则不允许种植
            if land_type not in crop_suitable_land[j]:
                for k in years:
                    for t in range(1, seasons[i] + 1):
                        model += x[i, j, k, t] == 0  # 不允许种植不适合的作物

# 求解模型
model.solve()

# 逐年计算目标函数值
for k in years:
    yearly_obj_value = lpSum((crop_prices[j] * crop_yields[j] - cost[j]) * x[i, j, k, t].varValue
                             for i in land_areas for j in crop_yields for t in range(1, seasons[i] + 1)
                             if x[i, j, k, t].varValue > 0)  # 只计算种植面积大于 0 的决策变量
    print(f"Year {k} 目标函数值: {yearly_obj_value}")

# 按年整理种植策略
yearly_planting_strategy = {}

# 遍历每一个决策变量，按年（k）进行汇总
for i in land_areas:
    for j in crop_yields:
        for k in years:
            for t in range(1, seasons[i] + 1):
                if x[i, j, k, t].varValue > 0:  # 只输出非零种植面积的策略
                    if k not in yearly_planting_strategy:
                        yearly_planting_strategy[k] = []
                    yearly_planting_strategy[k].append({
                        'land': i,
                        'crop_id': j,
                        'season': t,
                        'planting_area': x[i, j, k, t].varValue
                    })

# 按年份输出种植策略
for year, strategy in yearly_planting_strategy.items():
    print(f"Year {year}:")
    df_yearly = pd.DataFrame(strategy)
    print(df_yearly)


Year 2024 目标函数值: 20706540.0
Year 2025 目标函数值: 20781891.25
Year 2026 目标函数值: 20966535.0
Year 2027 目标函数值: 20294060.0
Year 2028 目标函数值: 21042087.5
Year 2029 目标函数值: 21118818.75
Year 2030 目标函数值: 18674806.25
Year 2025:
    land  crop_id  season  planting_area
0     A1        2       1           40.0
1     A1       20       1           10.5
2     A1       36       1           10.0
3     A1       28       1            0.3
4     A1       21       1            0.3
..   ...      ...     ...            ...
810   F1       40       1            0.6
811   F2       40       1            0.6
812   F3       19       1            0.3
813   F3       40       1            0.3
814   F4       40       1            0.6

[815 rows x 4 columns]
Year 2028:
    land  crop_id  season  planting_area
0     A1        2       1           40.0
1     A1       20       1           10.5
2     A1       36       1           10.0
3     A1       28       1            0.3
4     A1       21       1            0.3
..   ...      ...

In [19]:
import pandas as pd

# 读取用户上传的 result1_1.xlsx
result_file_path = 'result1_1.xlsx'
result_data_template = pd.read_excel(result_file_path)

# 定义作物编号与名称的映射（使用正确的列名）
crop_id_to_name = dict(zip(file1_data2['crop_id'], file1_data2['crop_name']))

# 创建一个函数来根据种植策略更新表格
def update_result_table(planting_strategy, result_df):
    for record in planting_strategy:
        land = record['land']  # 地块名称
        crop_id = record['crop_id']  # 作物编号
        season = record['season']  # 季节
        planting_area = record['planting_area']  # 种植面积
        
        # 将 crop_id 转换为 crop_name
        crop_name = crop_id_to_name.get(crop_id, None)
        
        # 找到地块对应的行
        row_index = result_df.index[result_df['地块名'] == land]
        if season == 1:
               # 将种植面积写入第一季的列
            result_df.loc[row_index, crop_name] = planting_area
        else:
                # 将种植面积写入第二季的列
            result_df.loc[row_index + 1, crop_name] = planting_area

# 创建一个字典存储每年的结果
yearly_result_data = {}

# 遍历每个年份，将每年的种植策略填入对应的表格
for year, strategy in yearly_planting_strategy.items():
    # 使用 result_data_template 作为基础
    result_data_year = result_data_template.copy()
    # 更新每年的数据
    update_result_table(strategy, result_data_year)
    # 将结果保存到字典中
    yearly_result_data[f'{year}'] = result_data_year

# 将所有年份的结果写入一个包含多个 sheet 的 Excel 文件
with pd.ExcelWriter('result1_1_multisheet.xlsx') as writer:
    for sheet_name, data in yearly_result_data.items():
        data.to_excel(writer, sheet_name=sheet_name, index=False)

print('成功')

成功


In [ ]:
第二种情况

In [21]:
import pandas as pd
from pulp import LpMaximize, LpProblem, LpVariable, lpSum, LpBinary

# 读取Excel文件中的数据
file1_path = '附件1.xlsx'
file2_path = '附件2_1.xlsx'

# 读取每个工作表的数据
file1_data1 = pd.read_excel(file1_path, sheet_name='乡村的现有耕地')
file1_data2 = pd.read_excel(file1_path, sheet_name='乡村种植的农作物')
file2_data1 = pd.read_excel(file2_path, sheet_name='Sheet1')
file2_data2 = pd.read_excel(file2_path, sheet_name='Sheet2')

# 清理列名
file1_data1.columns = ['land_name', 'land_type', 'land_area', 'description']
file1_data2.columns = ['crop_id', 'crop_name', 'crop_type', 'suitable_land', 'description']
file2_data1.columns = ['land_name', 'crop_id', 'crop_name', 'crop_type', 'planting_area', 
                      'planting_season', 'land_type', 'yield_per_mu', 'planting_period', 'expected_sales']
file2_data2.columns = ['serial_no', 'crop_id', 'crop_name', 'land_type', 'planting_season', 
                       'yield_per_mu', 'planting_cost_per_mu', 'sales_price_per_unit']

# 处理价格区间，将价格区间转换为均值
def parse_price_range(price_str):
    if isinstance(price_str, str) and '-' in price_str:
        low, high = map(float, price_str.split('-'))
        return (low + high) / 2
    return float(price_str)

file2_data2['average_price'] = file2_data2['sales_price_per_unit'].apply(parse_price_range)

# 使用 crop_id 作为键创建字典
crop_prices = dict(zip(file2_data2['crop_id'], file2_data2['average_price']))
crop_yields = dict(zip(file2_data1['crop_id'], file2_data1['yield_per_mu']))
expected_sales = dict(zip(file2_data1['crop_id'], file2_data1['expected_sales']))
cost = dict(zip(file2_data2['crop_id'], file2_data2['planting_cost_per_mu']))

# 创建土地面积的字典
land_areas = dict(zip(file1_data1['land_name'], file1_data1['land_area']))

# 定义每个地块类型对应的种植季节数
season_info = {
    '平旱地': 1,  # 每年只能种一季
    '梯田': 1,    # 每年只能种一季
    '山坡地': 1,  # 每年只能种一季
    '水浇地': 2,  # 每年可以种一季或两季
    '大棚': 2     # 每年可以种两季
}

# 根据 land_type 决定每块地每年可以种的季节数
seasons = {row['land_name']: season_info.get(row['land_type'], 1) for idx, row in file1_data1.iterrows()}

# 创建一个字典，存储每个作物可以种植的土地类型
crop_suitable_land = {}
for idx, row in file1_data2.iterrows():
    if pd.notna(row['suitable_land']):
        suitable_land_types = row['suitable_land'].split('\n')
        crop_suitable_land[row['crop_id']] = suitable_land_types

# 定义年份范围
years = list(range(2024, 2031))  # 从2024年到2030年，共7年

# 定义优化模型
model = LpProblem("Crop_Allocation_Optimization", LpMaximize)

# 决策变量的定义：种植面积不能为负
x = LpVariable.dicts("plant_area", 
                     [(i, j, k, t) for i in land_areas for j in crop_yields for k in years for t in range(1, seasons[i] + 1)], 
                     lowBound=0)  # lowBound=0 保证种植面积为非负数

# 二值变量 y 表示是否种植该作物
y = LpVariable.dicts("plant_decision", 
                     [(i, j, k, t) for i in land_areas for j in crop_yields for k in years for t in range(1, seasons[i] + 1)], 
                     cat=LpBinary)  # y 变量是二进制变量

# 引入辅助变量：sell_amount 表示正常销售量，overstock_amount 表示滞销量
sell_amount = LpVariable.dicts("sell_amount", [(i, j, k, t) for i in land_areas for j in crop_yields for k in years for t in range(1, seasons[i] + 1)], lowBound=0)
overstock_amount = LpVariable.dicts("overstock_amount", [(i, j, k, t) for i in land_areas for j in crop_yields for k in years for t in range(1, seasons[i] + 1)], lowBound=0)

# 目标函数：净利润 = 销售收入 - 种植成本
model += lpSum(
    crop_prices[j] * (sell_amount[i, j, k, t] + 0.5 * overstock_amount[i, j, k, t])  # 销售收入
    - cost[j] * x[i, j, k, t]  # 减去种植成本
    for i in land_areas for j in crop_yields for k in years for t in range(1, seasons[i] + 1)
)

# 约束：sell_amount <= 作物的实际产量和预期销售量的最小值
for i in land_areas:
    for j in crop_yields:
        for k in years:
            for t in range(1, seasons[i] + 1):
                model += sell_amount[i, j, k, t] <= crop_yields[j] * x[i, j, k, t]
                model += sell_amount[i, j, k, t] <= expected_sales[j]

# 约束：overstock_amount >= 作物的实际产量 - 预期销售量
for i in land_areas:
    for j in crop_yields:
        for k in years:
            for t in range(1, seasons[i] + 1):
                model += overstock_amount[i, j, k, t] >= crop_yields[j] * x[i, j, k, t] - expected_sales[j]
                model += overstock_amount[i, j, k, t] >= 0  # overstock 不能为负数

# 约束1：耕地面积约束 - 调整每块地每年每季种植面积的约束，确保每年不超过可用面积
for i in land_areas:
    for k in years:
        # 每年总种植面积不能超过该地块的总可用面积
        model += lpSum(x[i, j, k, t] for j in crop_yields for t in range(1, seasons[i] + 1)) <= land_areas[i]

# 约束2：每三年内至少种植一次豆类作物（假设豆类作物的 crop_id 为1）
for i in land_areas:
    model += lpSum(y[i, 1, k, t] for k in range(2024, 2027) for t in range(1, seasons[i] + 1)) >= 1

# 约束3：作物产量不得超过预期销售量
for i in land_areas:
    for j in crop_yields:
        for k in years:
            for t in range(1, seasons[i] + 1):
                model += crop_yields[j] * x[i, j, k, t] <= expected_sales[j]

# 约束4：同一地块不能在连续季节种植相同作物
for i in land_areas:
    for j in crop_yields:
        for k in years:
            for t in range(1, seasons[i]):  # 检查同一年不同季节
                model += y[i, j, k, t] + y[i, j, k, t+1] <= 1

# 约束5：最小种植面积约束（当 y[i, j, k, t] == 1，即种植某种作物时，才要求种植面积大于最小面积）
for i in land_areas:
    for j in crop_yields:
        for k in years:
            for t in range(1, seasons[i] + 1):
                # 如果种植某作物 (y[i, j, k, t] == 1)，种植面积必须 >= 0.5 * land_areas[i]
                model += x[i, j, k, t] >= 0.5 * land_areas[i] * y[i, j, k, t]

# 新增约束6：作物只能种植在适合的土地类型上
for i in land_areas:
    for j in crop_yields:
        land_type = file1_data1.loc[file1_data1['land_name'] == i, 'land_type'].values[0]
        # 先检查 crop_id 是否存在于 crop_suitable_land 字典中
        if j in crop_suitable_land:
            # 如果当前地块类型不适合该作物种植，则不允许种植
            if land_type not in crop_suitable_land[j]:
                for k in years:
                    for t in range(1, seasons[i] + 1):
                        model += x[i, j, k, t] == 0  # 不允许种植不适合的作物

# 求解模型
model.solve()

# 逐年计算净利润
for k in years:
    yearly_profit = lpSum(
        (crop_prices[j] * (sell_amount[i, j, k, t].varValue + 0.5 * overstock_amount[i, j, k, t].varValue)  # 销售收入
         - cost[j] * x[i, j, k, t].varValue)  # 减去种植成本
        for i in land_areas for j in crop_yields for t in range(1, seasons[i] + 1)
        if x[i, j, k, t].varValue > 0  # 只计算种植面积大于 0 的决策变量
    )
    print(f"Year {k} 净利润: {yearly_profit}")

# 按年整理种植策略
yearly_planting_strategy = {}

# 遍历每一个决策变量，按年（k）进行汇总
for i in land_areas:
    for j in crop_yields:
        for k in years:
            for t in range(1, seasons[i] + 1):
                if x[i, j, k, t].varValue > 0:  # 只输出非零种植面积的策略
                    if k not in yearly_planting_strategy:
                        yearly_planting_strategy[k] = []
                    yearly_planting_strategy[k].append({
                        'land': i,
                        'crop_id': j,
                        'season': t,
                        'planting_area': x[i, j, k, t].varValue
                    })

# 按年份输出种植策略
for year, strategy in yearly_planting_strategy.items():
    print(f"Year {year}:")
    df_yearly = pd.DataFrame(strategy)
    print(df_yearly)


Year 2024 净利润: 20607892.0
Year 2025 净利润: 20546782.0
Year 2026 净利润: 20153291.0
Year 2027 净利润: 21890080.0
Year 2028 净利润: 21890080.0
Year 2029 净利润: 21890080.0
Year 2030 净利润: 21890080.0
Year 2024:
    land  crop_id  season  planting_area
0     A1        1       1           15.0
1     A1       11       1            2.5
2     A1       13       1           18.0
3     A1       20       1           15.0
4     A1       36       1           10.0
..   ...      ...     ...            ...
850   F1        1       1            0.3
851   F1       40       1            0.3
852   F2       40       1            0.6
853   F3       40       1            0.6
854   F4       40       1            0.6

[855 rows x 4 columns]
Year 2025:
    land  crop_id  season  planting_area
0     A1        1       1           15.0
1     A1       11       1            2.5
2     A1       13       1           18.0
3     A1       20       1           15.0
4     A1       36       1           10.0
..   ...      ...     ...         

In [22]:
import pandas as pd

# 读取用户上传的 result1_1.xlsx
result_file_path = 'result1_1.xlsx'
result_data_template = pd.read_excel(result_file_path)

# 定义作物编号与名称的映射（使用正确的列名）
crop_id_to_name = dict(zip(file1_data2['crop_id'], file1_data2['crop_name']))

# 创建一个函数来根据种植策略更新表格
def update_result_table(planting_strategy, result_df):
    for record in planting_strategy:
        land = record['land']  # 地块名称
        crop_id = record['crop_id']  # 作物编号
        season = record['season']  # 季节
        planting_area = record['planting_area']  # 种植面积
        
        # 将 crop_id 转换为 crop_name
        crop_name = crop_id_to_name.get(crop_id, None)
        
        # 找到地块对应的行
        row_index = result_df.index[result_df['地块名'] == land]
        if season == 1:
               # 将种植面积写入第一季的列
            result_df.loc[row_index, crop_name] = planting_area
        else:
                # 将种植面积写入第二季的列
            result_df.loc[row_index + 1, crop_name] = planting_area

# 创建一个字典存储每年的结果
yearly_result_data = {}

# 遍历每个年份，将每年的种植策略填入对应的表格
for year, strategy in yearly_planting_strategy.items():
    # 使用 result_data_template 作为基础
    result_data_year = result_data_template.copy()
    # 更新每年的数据
    update_result_table(strategy, result_data_year)
    # 将结果保存到字典中
    yearly_result_data[f'{year}'] = result_data_year

# 将所有年份的结果写入一个包含多个 sheet 的 Excel 文件
with pd.ExcelWriter('result1_2_multisheet.xlsx') as writer:
    for sheet_name, data in yearly_result_data.items():
        data.to_excel(writer, sheet_name=sheet_name, index=False)

print('成功')

C:\Users\cyk\AppData\Roaming\Python\Python38\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


成功
